In [14]:
import os
import glob
from collections import OrderedDict

run_dir = '/projects/mmcpnnl/jlee/wrf-mmc/sgp/l3_0901m/'
dir_1 = 'z_to01-18z/'
dir_2 = 'z_to02-00z/'
dir_out = 'z_ts/'

# return list of duplicated indices in list
def list_duplicates(seq):
    seen = set()
    seen_add = seen.add
    # need to +1 because the first line of a ts file is not data
    return [idx+1 for idx, item in enumerate(seq) if item in seen or seen_add(item)]

files = [os.path.basename(i) for i in glob.glob(run_dir+dir_1+'*')]

# files = ['z.c1.d02.TH']

for f in files: 

    with open(run_dir+dir_1+f) as fp:
        data = fp.read()

    with open(run_dir+dir_2+f) as fp:
        # skip the first line because the first line of a ts file is not data
        next(fp)
        data2 = fp.read()

    data += data2

    list_all = list(OrderedDict.fromkeys(data.split('\n')))

    if f[-2:] == 'TS': 
        time_start, time_end = 6, 15
    else: 
        time_start, time_end = 4, 13
    
    # screen data based on time steps in hours
    time_steps = []
    for i in list_all[1:]:
        time_steps.append(i[time_start:time_end]) # hard coded...

    duplicate_index = list_duplicates(time_steps)

    list_out = [list_all[i] for i in range(len(list_all)) if i not in duplicate_index]

    list_out = '\n'.join(list_out)

    with open (run_dir+dir_out+f, 'w') as fp:
        fp.write(list_out)